In [1]:
import torch
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

In [2]:
# Define transforms (resize, normalize, convert to tensor)
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5]*3, std=[0.5]*3)  # Normalize RGB
])

# Load dataset
dataset = datasets.ImageFolder(root=r'G:\My Drive\Coding Stuff\PROJECTS\braintumour\archive\brain_tumor_dataset', transform=transform)

# Wrap in DataLoader
dataloader = DataLoader(dataset, batch_size=32, shuffle=True)

# Example: iterate through one batch
for images, labels in dataloader:
    print(images.shape)  # [32, 3, 224, 224]
    print(labels.shape)  # [32]
    break

torch.Size([32, 3, 224, 224])
torch.Size([32])


In [3]:
print(dataset.classes)
print(dataset.class_to_idx)

['no', 'yes']
{'no': 0, 'yes': 1}


In [ ]:
# Define and Train a Simple Classification Model
In this section, we will define a simple neural network and train it for classification.
</VSCode.Cell>
<VSCode.Cell language="python">
import torch.nn as nn
import torch.optim as optim
import torch
</VSCode.Cell>
<VSCode.Cell language="python">
# Define a simple neural network
class SimpleCNN(nn.Module):
    def __init__(self):
        super(SimpleCNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 16, kernel_size=3, stride=1, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.fc1 = nn.Linear(16 * 112 * 112, 2)
    def forward(self, x):
        x = self.pool(torch.relu(self.conv1(x)))
        x = x.view(-1, 16 * 112 * 112)
        x = self.fc1(x)
        return x
model = SimpleCNN()
</VSCode.Cell>
<VSCode.Cell language="python">
# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)
</VSCode.Cell>
<VSCode.Cell language="python">
# Training loop
epochs = 5
for epoch in range(epochs):
    for images, labels in dataloader:
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
    print(f"Epoch {epoch+1}/{epochs}, Loss: {loss.item()}")
